In [138]:
import configparser
from pathlib import Path
from datetime import datetime
from os import PathLike

import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', 0)

In [108]:
def read_config(fn):
    # reads in an ini config file, including any blank lines

    # read in file, add "Null" to any empty inputs
    fn = Path(fn)
    with open(fn, 'r') as f:
        lines = f.readlines()
    for i, _ in enumerate(lines):
        if lines[i][-2:] == '=\n':
            lines[i] = lines[i][:-1] + 'Null' + lines[i][-1]
    complete_fn = fn.parent / (fn.stem + '_complete.eddypro')
    with open(complete_fn, 'w') as f:
        for ln in lines:
            f.write(ln)

    ini = configparser.ConfigParser()
    ini.read(complete_fn)
    return ini

def read_config_as_dataframe(fn):
    lines = []
    ini = read_config(fn)
    for section in ini.sections():
        for option, value, in ini[section].items():
            lines.append([section, option, value])
    df = pd.DataFrame(lines, columns=['Section', 'Option', 'Value'])
    df = df.sort_values(['Section', 'Option'])
    df['Name'] = fn.stem
    return df

def compare_configs(df1: pd.DataFrame, df2: pd.DataFrame):
    # compare differences between two config dataframe
    df1_new = df1.loc[df1['Value'] != df2['Value'], ['Section', 'Option', 'Value']]
    df2_new = df2.loc[df1['Value'] != df2['Value'], ['Section', 'Option', 'Value']]
    name1 = df1['Name'].values[0]
    name2 = df2['Name'].values[0]
    df_compare = (
        df1_new
        .merge(df2_new, on=['Section', 'Option'], suffixes=['_'+name1, '_'+name2])
        .sort_values(['Section', 'Option'])
    )
    return df_compare

In [109]:
environment = Path('/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro')

# When we hit buttons in EddyPro, what settings change in the .ini file?


## Set up a template
First, we create a reference .eddypro file. This file will have all the default processing settings.

Note that EddyPro will leave some options blank

In [110]:
base = read_config_as_dataframe(environment / 'ini/base.eddypro')
base.head()

,Section,Option,Value,Name
0,FluxCorrection_SpectralAnalysis_General,add_sonic_lptf,1,base
1,FluxCorrection_SpectralAnalysis_General,ex_dir,Null,base
2,FluxCorrection_SpectralAnalysis_General,ex_file,Null,base
3,FluxCorrection_SpectralAnalysis_General,horst_lens,2,base
4,FluxCorrection_SpectralAnalysis_General,sa_bin_spectra,Null,base


## Basic Settings
### Changing dates
change **Basic-Settings/Start** and **Basic-Settings/End**

In [111]:
change_dates = read_config_as_dataframe(environment / 'ini/change_dates.eddypro')
compare_configs(base, change_dates)

,Section,Option,Value_base,Value_change_dates
0,Project,file_name,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/base.eddypro,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/change_dates.eddypro
1,Project,last_change_date,2023-07-26T10:48:05,2023-07-26T11:11:27
2,Project,pr_end_date,2020-07-22,2020-06-26
3,Project,pr_end_time,00:00,04:30
4,Project,pr_start_date,2020-06-21,2020-06-23
5,Project,pr_start_time,00:00,02:30
6,Project,project_id,base,change_dates


other than trivial changes and housekeeping changes like file_name, last_change_date, and project_id, the only changed settings are the start and end times and dates, as expected:

These are pretty self-explanatory:
* Project/pr_end_time
* Project/pr_start_time
* Project/pr_end_date
* Project/pr_start_date

### Missing Samples Allowance and Flux Averaging Interval
Change **Basic-Settings/Missing-Samples-Allowance** and **Basic-Settings/Flux-Averaging-Interval**

In [112]:
compare_configs(
    base, 
    read_config_as_dataframe(environment / 'ini/missing-avgint.eddypro')
)


,Section,Option,Value_base,Value_missing-avgint
0,Project,file_name,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/base.eddypro,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/missing-avgint.eddypro
1,Project,last_change_date,2023-07-26T10:48:05,2023-07-26T13:36:21
2,Project,project_id,base,missing-avgint
3,RawProcess_Settings,avrg_len,30,0
4,RawProcess_Settings,max_lack,10,0


Changes to INI file:
* RawProcess_Settings/avrg_len: averaging interval 
* RawProcess_Settings/max_lack: maximum % missing values

### North Reference
Change **Basic-Settings/North-Reference** to Geographic North, but don't change 

In [81]:
compare_configs(
    base, 
    read_config_as_dataframe(environment / 'ini/northref.eddypro')
)

,Section,Option,Value_base,Value_northref
0,Project,file_name,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/base.eddypro,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/northref.eddypro
1,Project,last_change_date,2023-07-26T10:48:05,2023-07-26T11:37:33
2,RawProcess_General,dec_date,2023-12-31,2020-07-22
3,RawProcess_General,mag_dec,0,8.666667
4,RawProcess_General,use_geo_north,0,1
5,RawProcess_TiltCorrection_Settings,pf_end_time,23:30,00:00


* RawProcess_General/dec_date (yyyy-mm-dd)
* RawProcess_General/mag_dec (decimal declination)
* RawProcess_General/use_geo_north (0 -> 1)
* TiltCorrection_Settings/pf_end_time (23:30 -> 00:00??, ignoring for now)

### Wind speed measurement offsets
Change **Advanced-Settings/Wind-Speed-Measurement-Offsets**

In [132]:
compare_configs(
    base, 
    read_config_as_dataframe(environment / 'ini/windspeedoffsets.eddypro')
)

,Section,Option,Value_base,Value_windspeedoffsets
0,Project,file_name,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/base.eddypro,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/windspeedoffsets.eddypro
1,Project,last_change_date,2023-07-26T10:48:05,2023-07-26T14:07:54
2,RawProcess_Settings,u_offset,0,1.0999999999999999
3,RawProcess_Settings,v_offset,0,-1.0999999999999999
4,RawProcess_Settings,w_offset,0,-0.30000000000000004


### Axis rotations for tilt correction
options are:

* Untick **Advanced-Settings/Processing-Options/Axis-Rotations-For-Tilt-Correction**
* 

In [133]:
compare_configs(
    base, 
    read_config_as_dataframe(environment / 'ini/norot.eddypro')
)

,Section,Option,Value_base,Value_norot
0,Project,file_name,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/base.eddypro,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/norot.eddypro
1,Project,last_change_date,2023-07-26T10:48:05,2023-07-26T14:14:00
2,Project,project_id,base,norot
3,RawProcess_Settings,rot_meth,1,0


In [134]:
compare_configs(
    base, 
    read_config_as_dataframe(environment / 'ini/tr.eddypro')
)

,Section,Option,Value_base,Value_tr
0,Project,file_name,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/base.eddypro,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/tr.eddypro
1,Project,last_change_date,2023-07-26T10:48:05,2023-07-26T14:15:11
2,Project,project_id,base,tr
3,RawProcess_Settings,rot_meth,1,2


In [137]:
compare_configs(
    base, 
    read_config_as_dataframe(environment / 'ini/pfnvb_with_file.eddypro')
)

,Section,Option,Value_base,Value_pfnvb_with_file
0,Project,file_name,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/base.eddypro,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/pfnvb_with_file.eddypro
1,Project,last_change_date,2023-07-26T10:48:05,2023-07-26T14:28:52
2,Project,project_id,base,pfnvb_with_file
3,RawProcess_Settings,rot_meth,1,4
4,RawProcess_TiltCorrection_Settings,pf_file,Null,/Users/alex/Documents/Work/UWyo/2023 Summer/Fluxcourse/Project/eddypro_output/lostcreek/Planar Fit Single-Sector/eddypro_template_planar_fit_2023-06-28T230545_adv.txt
5,RawProcess_TiltCorrection_Settings,pf_mode,1,0


Unticking tilt corrections: rot_meth=0

Selecting triple rotations: rot_meth=2

Selecting double rotations: rot_meth=1

Selecting planar fit with a provided file: rot_meth=3, RawProcess_TiltCorrection_Settings/pf_mode = 0, pf_file=filepath

Selecting planar fit (no v bias) with a provided file: rot_meth=4, RawProcess_TiltCorrection_Settings/pf_mode = 0, pf_file=filepath

In [141]:
compare_configs(
    base, 
    read_config_as_dataframe(environment / 'ini/pf_all_but_sectors_csectorclockwise.eddypro')
)

,Section,Option,Value_base,Value_pf_all_but_sectors_csectorclockwise
0,Project,file_name,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/base.eddypro,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/pf_all_but_sectors_csectorclockwise.eddypro
1,Project,last_change_date,2023-07-26T10:48:05,2023-07-26T14:49:42
2,Project,project_id,base,pf_all_but_sector_csectorclockwise
3,RawProcess_Settings,rot_meth,1,3
4,RawProcess_TiltCorrection_Settings,pf_end_date,2020-07-22,2020-07-23
5,RawProcess_TiltCorrection_Settings,pf_end_time,23:30,20:30
6,RawProcess_TiltCorrection_Settings,pf_fix,0,1
7,RawProcess_TiltCorrection_Settings,pf_min_num_per_sec,0,8
8,RawProcess_TiltCorrection_Settings,pf_start_date,2020-06-21,2020-03-03
9,RawProcess_TiltCorrection_Settings,pf_start_time,Null,05:00


When providing planar fit settings manually, change the following in RawProcess_TiltCorrection_Settings

* pf_start/end_date/time
* pf_subset: 1???
* pf_mode: 1 for manual, 0 for file
* pf_u_min: minimum mean horizontal windspeed [0, 10]
* pf_w_max: maximum mean vertical windspeed (0, 10]
* pf_min_num_per_sec: minimum elements per sector [0, 9999]
* pf_fix: what to do if calculations fail for a sector (closest clockwise 0, closest ccw 1, dr 2)
* pf_north_offset: north offset for minimum azimuth of first sector
* sectors: n goes from 1 to max of 10
  * pf_sector_n_exclude: 1 to ignore for PF, 0 to use for PF
  * pf_sector_n_width: degrees, from 1 to 359

### Extracting Turbulent Fluctuations
This one is in **Advanced-Settings/Processing/Turbulent-fluctuations** and is pretty simple:

RawProcess_Settings/detrend_meth: 0 (block avg), 1 (linear detrending), 2 (running mean), 3 (exponential running mean)

RawProcess_Settings/timeconst: 0 gives same as flux averaging interval, otherwise units are seconds.

### Time lags compensation


In [189]:
read_config_as_dataframe(environment / 'ini/tlags.eddypro').query('Section == "RawProcess_TimelagOptimization_Settings"')

,Section,Option,Value,Name
350,RawProcess_TimelagOptimization_Settings,to_ch4_max_lag,-999.3,tlags
351,RawProcess_TimelagOptimization_Settings,to_ch4_min_flux,0.200,tlags
352,RawProcess_TimelagOptimization_Settings,to_ch4_min_lag,-999.4,tlags
353,RawProcess_TimelagOptimization_Settings,to_co2_max_lag,-998.3,tlags
354,RawProcess_TimelagOptimization_Settings,to_co2_min_flux,2.000,tlags
355,RawProcess_TimelagOptimization_Settings,to_co2_min_lag,-998.4,tlags
356,RawProcess_TimelagOptimization_Settings,to_end_date,2020-07-22,tlags
357,RawProcess_TimelagOptimization_Settings,to_end_time,00:00,tlags
358,RawProcess_TimelagOptimization_Settings,to_file,/Users/alex/Documents/Work/UWyo/2023 Summer/Fluxcourse/Project/eddypro_output/lostcreek/Cov Max Auto Opt/eddypro_TimeLag-AutomaticOpt_timelag_opt_2023-06-28T220023_adv.txt,tlags
359,RawProcess_TimelagOptimization_Settings,to_gas4_max_lag,-999.0,tlags


In [190]:
compare_configs(
    base, 
    read_config_as_dataframe(environment / 'ini/tlags.eddypro')
)

,Section,Option,Value_base,Value_tlags
0,Project,file_name,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/base.eddypro,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/tlags.eddypro
1,Project,last_change_date,2023-07-26T10:48:05,2023-07-26T19:05:49
2,Project,project_id,base,tlags
3,RawProcess_Settings,out_bin_sp,0,1
4,RawProcess_Settings,tlag_meth,2,4
5,RawProcess_TimelagOptimization_Settings,to_ch4_max_lag,-1000.1,-999.3
6,RawProcess_TimelagOptimization_Settings,to_ch4_min_flux,0.200,0.202
7,RawProcess_TimelagOptimization_Settings,to_ch4_min_lag,-1000.1,-999.4
8,RawProcess_TimelagOptimization_Settings,to_co2_max_lag,-1000.1,1000.0
9,RawProcess_TimelagOptimization_Settings,to_co2_min_flux,2.000,1.998


RawProcess_Settings/tlag_meth:
* No compensation: tlag_meth = 0
* Constant: tlag_meth = 1, as specified in metadata
* Covariance maximization with default: 2
* Naive maximization: 3
* Automatic time lag optimization: provide a file or manually modify settings: 4
    * provide a file:
      * RawProcess_TimeLagOptimization_Settings to_file = path to timelag_opt output file
      * RawProcess_TimeLagOptimization_Settings to_mode = 0
      * RawProcess_TimeLagOptimization_Settings to_subset = 0
    * manually input settings:
      * RawProcess_TimeLagOptimization_Settings to_subset = 0 or 1?????
      * RawProcess_TimeLagOptimization_Settings to_mode = 1
      * to_co2/ch4/h2o/gas4_max/min_lag: -1000s to +1000s, max > min
      * to_start/end_date/time
      * etc...(FINISH)



In [170]:
import configparser
from pathlib import Path
from typing import Literal
import datetime
from os import PathLike
from collections.abc import Sequence

class eddypro_ConfigParser(configparser.ConfigParser):
    '''a child class of configparser.ConfigParser added methods to modify eddypro-specific settings'''
    def __init__(self):
        configparser.ConfigParser.__init__(self)

    def set_StartDate(
        self,
        start: str | datetime.datetime | None = None, 
    ):
        if isinstance(start, str):
            pr_start_date, pr_start_time = start.split(' ')
        else:
            pr_start_date = start.strftime(r'%Y-%m-%d')
            pr_start_time = start.strftime(r'%H:%M')
        
        if start is not None:
            self.set(section='Project', option='pr_start_date', value=str(pr_start_date))
            self.set(section='Project', option='pr_start_time', value=str(pr_start_time))
    
    def set_EndDate(
        self,
        end: str | datetime.datetime | None = None
    ):
        """format yyyy-mm-dd HH:MM for strings"""
        if isinstance(end, str):
            pr_end_date, pr_end_time = end.split(' ')
        else:
            pr_end_date = end.strftime(r'%Y-%m-%d')
            pr_end_time = end.strftime(r'%H:%M')
        if end is not None:
            self.set(section='Project', option='pr_end_date', value=str(pr_end_date))
            self.set(section='Project', option='pr_end_time', value=str(pr_end_time))
        
    def set_DateRange(
        self,
        start: str | datetime.datetime | None = None, 
        end: str | datetime.datetime | None = None
    ):
        """format yyyy-mm-dd HH:MM for strings"""
        self.set_start_date(start)
        self.set_end_date(end)

    def set_MissingSamplesAllowance(self, pct: int):
        # pct: value from 0 to 40%
        assert pct >= 0 and pct <= 40
        self.set(section='RawProcess_Settings', option='max_lack', value=str(int(pct)))
    
    def set_FluxAveragingInterval(self, minutes: int):
        """minutes: how long to set the averaging interval to. If 0, use the file as-is"""
        assert minutes >= 0 and minutes <= 9999, 'Must have 0 <= minutes <= 9999'
        self.set(section='RawProcess_Settings', option='avrg_len', value=str(int(minutes)))
    
    def set_NorthReference(
        self, 
        method: Literal['mag', 'geo'], 
        magnetic_declination: float | None = None, 
        declination_date: str | datetime.datetime | None = None,
    ):
        """set the north reference to either magnetic north (mag) or geographic north (geo). If geographic north, then you must provide a magnetic delcination and a declination date.
        
        method: one of 'mag' or 'geo'
        magnetic_declination: a valid magnetic declination as a real number between -90 and 90. If 'geo' is selected, magnetic declination must be provided. Otherwise, does nothing.
        declination_date: the reference date for magnetic declination, either as a yyyy-mm-dd string or as a datetime.datetime object. If method = 'geo', then declination date must be provided. Otherwise, does nothing.
        """

        assert method in ['mag', 'geo'], "Method must be one of 'mag' (magnetic north) or 'geo' (geographic north)"

        self.set(section='RawProcess_General', option='use_geo_north', value=str(int(method == 'geo')))
        if method == 'geo':
            assert magnetic_declination is not None and declination_date is not None, 'declination and declination date must be provided if method is "geo."'
            assert magnetic_declination >= -90 and magnetic_declination <= 90, "Magnetic declination must be between -90 and +90 (inclusive)"
            self.set(section='RawProcess_General', option='mag_dec', value=str(round(magnetic_declination, 3)))
            if isinstance(declination_date, str):
                declination_date, _ = declination_date.split(' ')
            else:
                declination_date = declination_date.strftime(r'%Y-%m-%d')
            self.set(section='RawProcess_General', option='dec_date', value=str(declination_date))

    def set_ProjectId(self, project_id: str):
        assert ' ' not in project_id and '_' not in project_id, 'project id must not contain spaces or underscores.'
        self.set(section='Project', option='project_id', value=str(project_id))
    
    def set_WindSpeedMeasurementOffsets(self, u: float | None = None, v: float | None = None, w: float | None = None):
        assert max(u**2, v**2, w**2) <= 100, 'Windspeed measurement offsets cannot exceed ±10m/s'
        if u is not None:
            self.set(section='RawProcess_Settings', option='u_offset', value=str(round(u, 1)))
        if v is not None:
            self.set(section='RawProcess_Settings', option='v_offset', value=str(round(v, 1)))
        if w is not None:
            self.set(section='RawProcess_Settings', option='w_offset', value=str(round(w, 1)))
        
    def configure_PlanarFitSettings(
        self,
        w_max: float,
        u_min: float = 0,
        start: str | datetime.datetime | None = None,
        end: str | datetime.datetime | None = None,
        num_per_sector_min: int = 0,
        fix_method: Literal['CW', 'CCW', 'DR'] | int = 'CW',
        north_offset: int = 0,
        sectors: Sequence[Sequence[bool | int, float]] | None  = None,
    ) -> dict:
        """outputs a dictionary of planarfit settings
        w_max: the maximum mean vertical wind component for a time interval to be included in the planar fit estimation
        u_min: the minimum mean horizontal wind component for a time interval to be included in the planar fit estimation
        start, end: start and end date-times for planar fit computation. If a string, must be in yyyy-mm-dd HH:MM format. If None (default), set to the date range of the processing file.
        num_per_sector_min: the minimum number of valid datapoints for a sector to be computed. Default 0.
        fix_method: one of CW, CCW, or DR or 0, 1, 2. The method to use if a planar fit computation fails for a given sector. Either next valid sector clockwise, next valid sector, counterclockwise, or double rotations. Default is next valid sector clockwise.
        north_offset: the offset for the counter-clockwise-most edge of the first sector in degrees from -180 to 180. Default 0.
        sectors: list of tuples of the form (exclude/keep, width). Where exclude/keep is either a bool (False, True), or an int (0, 1) indicating whether to ingore this sector entirely when estimating planar fit coefficients. Width is a float between 0.1 and 359.9 indicating the width, in degrees of a given sector. Widths must add to one. If None (default), provide no sector information.

        Returns: a dictionary to provide to set_AxisRotationsForTiltCorrection
        """

        # start/end date/time
        if start is not None:
            if isinstance(start, str):
                pf_start_date, pf_start_time = start.split(' ')
            else:
                pf_start_date = start.strftime(r'%Y-%m-%d')
                pf_start_time = start.strftime(r'%H:%M')
        else:
            pf_start_date = self.get(section='Project', option='pr_start_date')
            pf_start_time = self.get(section='Project', option='pr_start_time')
        if end is not None:
            if isinstance(end, str):
                    pf_end_date, pf_end_time = end.split(' ')
            else:
                pf_end_date = end.strftime(r'%Y-%m-%d')
                pf_end_time = end.strftime(r'%H:%M')
        else:
            pf_end_date = self.get(section='Project', option='pr_end_date')
            pf_end_time = self.get(section='Project', option='pr_end_time')

        # simple settings
        assert u_min >= 0 and u_min <= 10, 'must have 0 <= u_min <= 10'
        assert w_max > 0 and w_max <= 10, 'must have 0 < w_max <= 10'
        assert isinstance(num_per_sector_min, int) and num_per_sector_min >= 0 and num_per_sector_min <= 9999, 'must have 0 <= num_sectors_min <= 9999'
        assert fix_method in ['CW', 'CCW', 'DR', 0, 1, 2], 'fix method must be one of CW, CCW, DR, 0, 1, 2'
        fix_dict = dict(CW = 0, CCW=1, DR=2)
        if isinstance(fix_method, str):
            fix_method = fix_dict[fix_method]

        assert north_offset >= -179.9 and north_offset <= 180, 'must have -179.9 <= north_offset <= 180'

        settings_dict = dict(
            pf_start_date=pf_start_date,
            pf_start_time=pf_start_time,
            pf_end_date=pf_end_date,
            pf_end_time=pf_end_time,
            pf_u_min=round(u_min, 1),
            pf_w_max=round(w_max, 1),
            pf_min_num_per_sec=int(num_per_sector_min),
            pf_fix=int(fix_method),
            pf_north_offset=round(north_offset, 1),
        )

        # sectors
        if sectors is not None:
            assert len(sectors) <= 10, "Can't have more than 10 sectors"
            total_width = 0
            for _, width in sectors:
                total_width += width
            assert total_width <= 360, 'Sector widths cannot add up to more than 360.'
            for i, sector in enumerate(sectors):
                exclude, width = sector
                n = i + 1
                settings_dict[f'pf_sector_{n}_exclude'] = int(exclude)
                settings_dict[f'pf_sector_{n}_width'] = str(round(width, 1))
        
        return settings_dict

    def set_AxisRotationsForTiltCorrection(
            self, 
            method: Literal['None', 'DR', 'TR', 'PF', 'PFNVB'] | int,
            pf_file: str | PathLike[str] | None = None,
            pf_settings_kwargs: dict | None = None,
        ):
        """
        method: one of 0 or "None" (no tilt correction), 1 or "DR" (double rotations), 2 or "TR" (triple rotations), 3 or "PF" (planar fit, Wilczak 2001), 4 or "PFNVB" (planar with with no velocity bias (van Dijk 2004)). one of pf_file or pf_settings must be provided if method is a planar fit type.
        pf_file: Mututally exclusive with pf_settings. If method is a planar fit type, path to an eddypro-compatible planar fit file. This can be build by hand, or taken from the output of a previous eddypro run. Typically labelled as "eddypro_<project id>_planar_fit_<timestamp>_adv.txt"
        pf_settings_kwargs: Mututally exclusive with pf_file. Arguments to be passed to configure_PlanarFitSettings.
        """
        method_dict = {'None':0, 'DR':1, 'TR':2, 'PF':3, 'PFNVB':4}
        if isinstance(method, str):
            assert method in ['None', 'DR', 'TR', 'PF', 'PFNVB'], 'method must be one of None, DR, TR, PF, PFNVB, or 0, 1, 2, 3, or 4.'
            method = method_dict[method]
        assert method in range(5), 'method must be one of None, DR, TR, PF, PFNVB, or 0, 1, 2, 3, or 4.'

        self.set(section='RawProcess_Settings', option='rot_meth', value=str(method))

        # planar fit
        if method in [3, 4]:
            assert bool(pf_file) != bool(pf_settings), 'If method is a planar-fit type, exactly one of pf_file or pf_settings should be specified.'
            if pf_file is not None:
                self.set(section='RawProcess_TiltCorrection_Settings', option='pf_file', value=str(pf_file))
                self.set(section='RawProcess_TiltCorrection_Settings', option='pf_mode', value=str(0))
                self.set(section='RawProcess_TiltCorrection_Settings', option='pf_subset', value=str(1))
            elif pf_settings_kwargs is not None:
                self.set(section='RawProcess_TiltCorrection_Settings', option='pf_file', value=str(pf_file))
                self.set(section='RawProcess_TiltCorrection_Settings', option='pf_mode', value=str(1))
                self.set(section='RawProcess_TiltCorrection_Settings', option='pf_subset', value=str(1))
                pf_settings = self.configure_PlanarFitSettings(**pf_settings_kwargs)
                for option, value in pf_settings.items():
                    self.set(section='RawProcess_TiltCorrection_Settings', option=option, value=str(value))
            
    def set_TurbulentFluctuations(self, method: Literal['block', 'detrend', 'rmean', 'exrmean'] | int = 0, time_const: float | None = None):
        '''time constant in seconds not required for block averaging (0) (default)'''
        method_dict = {'block':0, 'detrend':1, 'rmean':2, 'exrmean':3}
        if isinstance(method, str):
            assert method in method_dict, 'method must be one of block, detrend, rmean, exrmean'
            method = method_dict[method]
        if time_const is None:
            # default for linear detrend is flux averaging interval
            if method == 1:
                time_const = 0.
            # default for linear detrend is 250s
            elif method in [2, 3]:
                time_const = 250.
        self.set(section='RawProcess_Settings', option='detrend_meth', value=str(method))
        self.set(section='RawProcess_Settings', option='timeconst', value=str(time_const))
        
        
    
    def to_eddypro(self, fn):
        "write to a .eddypro file"
        self.set(section='Project', option='file_name', value=str(fn))
        with open(fn, 'w') as configfile:
            configfile.write(';EDDYPRO_PROCESSING\n')  # header line
            self.write(fp=configfile, space_around_delimiters=False)


{'pf_start_date': '2023-01-01', 'pf_start_time': '03:10', 'pf_end_date': '2023-01-10', 'pf_end_time': '10:10', 'pf_u_min': 5.1, 'pf_w_max': 9.1, 'pf_min_num_per_sec': 50, 'pf_fix': 0, 'pf_north_offset': 30.2, 'pf_sector_1_exclude': 1, 'pf_sector_1_width': '30', 'pf_sector_2_exclude': 1, 'pf_sector_2_width': '30', 'pf_sector_3_exclude': 1, 'pf_sector_3_width': '30', 'pf_sector_4_exclude': 1, 'pf_sector_4_width': '30', 'pf_sector_5_exclude': 1, 'pf_sector_5_width': '30', 'pf_sector_6_exclude': 1, 'pf_sector_6_width': '30', 'pf_sector_7_exclude': 1, 'pf_sector_7_width': '30', 'pf_sector_8_exclude': 1, 'pf_sector_8_width': '30', 'pf_sector_9_exclude': 1, 'pf_sector_9_width': '30', 'pf_sector_10_exclude': 0, 'pf_sector_10_width': '15'}
